In [35]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !pip install -q qiskit==1.2.4
    !pip install -q docplex
    !pip install -q cplex==22.1.1.2
    !pip install -q pyqubo
    !pip install -q dwave-ocean-sdk==6.10.0
    !pip install -q qiskit==1.2.4
    !pip install -q qiskit-aer==0.15.1
    !pip install -q qiskit-algorithms==0.3.1
    !pip install -q qiskit-optimization==0.6.1
    !pip install -q qiskit-ibm-runtime==0.31.0
    print("All Packages Installed!")

All Packages Installed!


In [36]:
# Define small input
weights_small = [8, 2, 3, 7]  # Weights of items (4 items)
bin_capacity_small = 10       # Capacity of each bin

# Define medium input
weights_medium = [5, 8, 3, 7, 6, 2, 4, 9, 3, 1]  # Weights of items (10 items)
bin_capacity_medium = 15        # Capacity of each bin

# Define large input
weights_large = [5, 8, 3, 7, 6, 2, 4, 9, 3, 1, 10, 2, 5, 8, 4, 3, 7, 1, 6, 9,
                3, 5, 2, 7, 8, 6, 4, 5, 6, 3, 2, 7, 9, 8, 3, 4, 5, 2, 1, 10,
                5, 7, 3, 6, 2, 8, 9, 4, 7, 1]  # Weights of items (50 items)
bin_capacity_large = 20         # Capacity of each bin

# Section 1: Classical Techniques

We demonstrate how to solve the Bin Packing problem (BPP) using its Integer Linear Programming (ILP) formulation. We use DOcplex to model the problem, and then call the IBM ILOG CPLEX Optimizer to solve it.

Given the parameters than:
*   $w_i$ :  Weight of item $i$
*   $C$ : Capacity of each bin
*   $n$ : Number of items
*   $m$ : Number of bins

and decisions variables:
*   $x_{ij} \in \{0, 1\}$: Binary variable that equals $1$ if item $i$ is placed in bin $j$, and $0$ otherwise.
*   $y_j \in \{0, 1\}$: Binary variable that equals $1$ if bin $j$ is used, and $0$ otherwise.




The ILP formulation of BPP is:

**Objective**
$$\min \sum_{j=1}^{m} y_j$$

**Constraints**
$$\sum_{i=1}^{n} w_i x_{ij} \leq C \cdot y_j \quad \forall j \in \{1, 2, \dots, m\}
   $$

$$\sum_{j=1}^{m} x_{ij} = 1 \quad \forall i \in \{1, 2, \dots, n\}$$

In the following code, we formulate the ILP using DOcplex and solve using CPLEX, while returning the DOcplex model.

In [37]:
from docplex.mp.model import Model

In [38]:
def solve_ilp_bpp(weights, bin_capacity):
    """
    Solves the Bin Packing Problem (BPP) using Integer Linear Programming (ILP) with DOcplex.

    The function formulates a bin packing problem where each item has a weight, and the goal
    is to minimize the number of bins required to pack all items. The bin capacity is fixed,
    and each item must be packed into one bin, such that the total weight of items in any bin
    does not exceed the bin capacity. It solves the model with optimal outputs the model to
    the user for future use.

    Args:
        weights (list of int/float): List of item weights to be packed.
        bin_capacity (int/float): The capacity of each bin.

    Returns:
        docplex.mp.model.Model: The optimized model object.
    """
    n_items = len(weights)  # Number of items to pack
    n_bins = n_items        # Maximum bins needed (one bin per item in the worst case)

    # Initialize the optimization model
    model = Model("Bin Packing")

    # Define binary variables: x[i][j] = 1 if item i is placed in bin j, else 0
    x = {(i, j): model.binary_var(name=f"x_{i}_{j}") for i in range(n_items) for j in range(n_bins)}

    # Define binary variables: y[j] = 1 if bin j is used, else 0
    y = {j: model.binary_var(name=f"y_{j}") for j in range(n_bins)}

    # Objective: Minimize the number of bins used
    model.minimize(model.sum(y[j] for j in range(n_bins)))

    # Constraint 1: Bin capacity should not be exceeded
    for j in range(n_bins):
        model.add_constraint(
            model.sum(weights[i] * x[i, j] for i in range(n_items)) <= bin_capacity * y[j],
            f"bin_capacity_{j}"
        )

    # Constraint 2: Each item must be placed in exactly one bin
    for i in range(n_items):
        model.add_constraint(
            model.sum(x[i, j] for j in range(n_bins)) == 1, f"item_{i}"
        )

    # Solve the model to find the optimal solution
    solution = model.solve()

    # If a solution is found, print the results
    if solution:
        print("Minimum number of bins used:", solution.get_objective_value())
        for j in range(n_bins):
            if y[j].solution_value > 0.5:  # Bin j is used
                print(f"Bin {j+1} contains items:", end=" ")
                for i in range(n_items):
                    if x[i, j].solution_value > 0.5:  # Item i is in bin j
                        print(f"item {i+1} (weight {weights[i]})", end=" ")
                print()
    else:
        print("No solution found")

    # Return the model object for further analysis or manipulation if needed
    return model

In [39]:
bpp_ilp_model = solve_ilp_bpp(weights_small, bin_capacity_small)

Minimum number of bins used: 2.0
Bin 1 contains items: item 1 (weight 8) item 2 (weight 2) 
Bin 2 contains items: item 3 (weight 3) item 4 (weight 7) 


# Section 2: Solving Quadratic Unconstrained Binary Optimization Problems with DWAVE

In [40]:
from pyqubo import Array, Constraint

We then demonstrate how to convert our BPP ILP instance to a Quadratic Unconstrained Binary Optimization (QUBO) format. Note we are lucky that the original ILP only has binary variables, thus we don't need to perform any binarization on the original variables.

Unfortunately, we still have inequalities and thus will need to introduce slack variables which we will need in binarize. Thus our modify ILP with only inequalities and slack variables becomes:

$$\min \sum_{j=1}^{m} y_j$$

$$\sum_{i=1}^{n} w_j x_{ij} + s_{i} = C \cdot y_j \quad \forall j \in \{1, 2, \dots, m\}
   $$

$$\sum_{j=1}^{m} x_{ij} = 1 \quad \forall i \in \{1, 2, \dots, n\}$$

Considering we can weakly bound $s_{i} \leq C$, we can binarize $s_{i}$ using the set of powers of $2$ less than $C$ (assume this set has size of $B$). Thus we introduce a new set of binary decision variables ${b_{ij}}$ such that $\sum_{i=1}^{B} 2^{i}b_{ij} = s_{j}$ where $b_{ij} = 1$, if $b_{ij}$ is $1$ if the $2^{i}$ is present in the binary decomposition of $s_{j}$. Thus the revised QUBO for this binarization on slack variable is:

$$\min \sum_{j=1}^{m} y_j$$

$$\sum_{i=1}^{n} w_j x_{ij} + \sum_{i=1}^{B} 2^{i}b_{ij} = C \cdot y_j \quad \forall j \in \{1, 2, \dots, m\}
   $$

$$\sum_{j=1}^{m} x_{ij} = 1 \quad \forall i \in \{1, 2, \dots, n\}$$

Finally, we use Lagrange multipliers to add each constraint to the objective. Thus our final QUBO model is

$$ H = \min \sum_{j=1}^{m} y_j + λ_{1}\sum_{j=1}^{m}(C \cdot y_{j} - (\sum_{i=1}^{n} w_{j}x_{ij} + \sum_{i=1}^{B} 2^{i}b_{ij}))^{2} + λ_{2}\sum_{i=1}^{n}(1 - \sum_{j=1}^{m}x_{ij})^{2}$$

Now while this is the true QUBO formulation of BPP which incorporates all key principles of binarization, slack variables, and lagrange multipliers, below I implemented a simpler (not completely accurate version).

For the code, I exclude the slack variable and the resulting binarization for demonstration purposes. If these were included, the brute-force QUBO solver would never work as it runs exponentially long with respect to the number of variables. Binarization introduces significantly more variables and thus even the smallest test case would be unfeasible to run.

Instead, the QUBO implemented treats the original inequality constraint and an equality. It performs roughly the same across the provided tests and is list below:

$$ H = \min \sum_{j=1}^{m} y_j + λ_{1}\sum_{j=1}^{m}(C \cdot y_{j} - (\sum_{i=1}^{n} w_{j}x_{ij}))^{2} + λ_{2}\sum_{i=1}^{n}(1 - \sum_{j=1}^{m}x_{ij})^{2}$$

Note the the QUBO were constructed using PyQUBO, and the function below returns the pyqubo model which can then either be passed into a quantum annealing solver or a brute force solver.

In [41]:
def get_qubo(weights, bin_capacity, single_assignment_penalty=10, bin_capacity_penalty=10):
    """
    Formulates the Bin Packing Problem (BPP) as a Quadratic Unconstrained Binary Optimization (QUBO) model.

    The function constructs a QUBO model for solving the Bin Packing Problem using quantum optimization techniques
    or classical solvers that support QUBO formulations. The model minimizes the number of bins used while enforcing
    that each item is assigned to exactly one bin, and that the total weight in each bin does not exceed its capacity.

    Args:
        weights (list of int/float): List of item weights to be packed.
        bin_capacity (int/float): The capacity of each bin.
        single_assignment_penalty (int, optional): Penalty for violating the "one item per bin" constraint. Default is 10.
        bin_capacity_penalty (int, optional): Penalty for violating the "bin capacity" constraint. Default is 10.

    Returns:
        model: The compiled Pyqubo model representing the QUBO formulation for the Bin Packing Problem.
    """

    # Number of items and bins
    n_items = len(weights)           # Total number of items
    n_bins = n_items                 # Maximum bins needed (one bin per item in the worst case)

    # Define binary variables using Pyqubo's Array
    # x[i][j] = 1 if item i is placed in bin j, 0 otherwise
    x = Array.create('x', shape=(n_items, n_bins), vartype='BINARY')

    # y[j] = 1 if bin j is used, 0 otherwise
    y = Array.create('y', shape=n_bins, vartype='BINARY')

    # Objective: Minimize the number of bins used
    H_obj = sum(y[i] for i in range(n_bins))  # Sum of y[j] over all bins

    # Constraint 1: Each item must be placed in exactly one bin
    # We square the difference to penalize violations
    H_single_assignment = sum((sum(x[i][j] for j in range(n_bins)) - 1)**2 for i in range(n_items))

    # Constraint 2: Bin capacity constraint
    # We square the difference to penalize violations of bin capacity
    H_capacity = sum((sum(weights[i] * x[i][j] for i in range(n_items)) - bin_capacity * y[j])**2 for j in range(n_bins))

    # Combine the objective and the penalties for constraints into one Hamiltonian
    H = H_obj + single_assignment_penalty * H_single_assignment + bin_capacity_penalty * H_capacity

    # Compile the QUBO model
    model = H.compile()

    return model


In [42]:
bpp_qubo_model = get_qubo(weights_small, bin_capacity_small)

## Brute Force

We first provide a brute-force solver which only works on small QUBOs. Unlike every other algorithm provided in this notebook, this is an exact solver and not just a heuristic.

Unfortunately it is exponentially slow, as for $n$ decision variables, it just iterates over all $2^{n}$ decision variable combinations for the best solution (i.e. lowest value in the QUBO). This function may take some time to run, even with the smallest test case.

In [43]:
import itertools

In [44]:
def brute_force_qubo_solver(model):
    """
    Solves a QUBO problem using brute-force search.

    This function performs a brute-force search over all possible binary assignments of the
    decision variables and evaluates the corresponding energy (objective value). The
    assignment with the minimum energy is returned as the best solution.

    Args:
        model: A QUBO model object that contains the decision variables and methods
               for evaluating the energy of a given assignment.

    Returns:
        best_assignment (dict): The binary assignment of variables that minimizes the energy.
        best_energy (float): The minimum energy (objective value) corresponding to the best assignment.
    """

    # Retrieve the list of variables from the QUBO model
    variables = model.variables
    num_variables = len(variables)  # Total number of variables in the model

    # Initialize the best assignment and energy
    best_assignment = {}  # Will store the binary assignment with the lowest energy
    best_energy = float('inf')  # Set initial best energy to infinity (indicating no solution found yet)

    # Iterate over all possible binary assignments (2^num_variables possibilities)
    count = 0
    for binary_tuple in itertools.product([0, 1], repeat=num_variables):
        # Create a dictionary from the binary tuple where the keys are the variables
        variable_assignment_dict = dict(zip(variables, binary_tuple))

        # Decode the binary assignment and calculate its energy using the model's decode_sample method
        decoded_sample = model.decode_sample(variable_assignment_dict, vartype='BINARY')

        # If the energy of the current sample is lower (better) than the previous best, update the best assignment
        if decoded_sample.energy < best_energy:
            best_assignment = decoded_sample.sample  # Save the new best assignment
            best_energy = decoded_sample.energy  # Save the energy of the new best assignment

    return best_assignment, best_energy

In [45]:
def parse_qubo_solution(binary_assignment, weights):
    """
    Parses the binary assignment solution for the Bin Packing Problem and prints the bin assignments.

    Args:
        binary_assignment (dict): Dictionary containing the binary variable assignments (e.g., 'x[i][j]' and 'y[j]').
        num_items (int): The number of items to pack.
        num_bins (int): The number of bins.
        weights (list): A list of item weights.

    Prints:
        - The minimum number of bins used.
        - Which items are assigned to which bins.
    """
    # Initialize a list to track which bins are used
    num_items = len(weights)
    num_bins = num_items
    used_bins = [False] * num_bins

    # Check which bins are used based on 'y[j]' values
    for j in range(num_bins):
        if binary_assignment.get(f'y[{j}]', 0) > 0.5:  # Bin j is used
            used_bins[j] = True

    # Print the minimum number of bins used
    bins_used = sum(used_bins)
    print(f"Minimum number of bins used: {bins_used}")

    # Iterate over bins and print the items assigned to each bin
    for j in range(num_bins):
        if used_bins[j]:
            print(f"Bin {j+1} contains items:", end=" ")
            for i in range(num_items):
                if binary_assignment.get(f'x[{i}][{j}]', 0) > 0.5:  # Item i is in bin j
                    print(f"item {i+1} (weight {weights[i]})", end=" ")
            print()  # Newline after printing items in the bin

In [46]:
best_assignment, best_energy = brute_force_qubo_solver(bpp_qubo_model)
parse_qubo_solution(best_assignment, weights_small)
print(f"Minimum energy (number of bins): {best_energy}")

Minimum number of bins used: 2
Bin 1 contains items: item 3 (weight 3) item 4 (weight 7) 
Bin 2 contains items: item 1 (weight 8) item 2 (weight 2) 
Minimum energy (number of bins): 2.0


In [47]:
import os
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import neal

# Connecting to DWAVE Account
os.environ['DWAVE_API_TOKEN'] = "DEV-7aecbd09552afc1cb9cba26c01212e6224025c2c"

Below, we demonstrate how to solve a QUBO using D-Wave's annealers. D-Wave offers both a built-in simulated annealing procedure and access to quantum annealers for solving QUBOs.

While the algorithms involved are too complex to cover in detail here, please refer to my previous paper for more information:
https://arulrhikm.github.io/media/documents/Solving_QUBOs_INFORMS_Version_for_Reviews.pdf


You can choose which annealing algorithm to use for your QUBO by passing an additional boolean argument. If the argument is false or not provided, simulated annealing will be used by default; otherwise, quantum annealing will be employed.

In [48]:
from dwave.system import EmbeddingComposite, DWaveSampler
import neal

def solve_qubo_annealing(model, quantum=False):
    """
    Solves a QUBO problem using either quantum annealing (via D-Wave) or simulated annealing.

    This function takes a QUBO model, converts it to a Binary Quadratic Model (BQM),
    and then solves it using the selected annealing method. It supports both quantum
    annealing (using D-Wave's quantum annealers) and classical simulated annealing.

    Args:
        model (pyqubo.Model): The QUBO model to be solved.
            The model will be converted to a Binary Quadratic Model (BQM).
        quantum (bool, optional): If `True`, quantum annealing will be used via D-Wave.
            If `False` or not provided, simulated annealing will be used. Default is `False`.

    Returns:
        dict: A dictionary representing the best solution found, where keys are the
              variables and values are the corresponding binary values (0 or 1).
    """

    # Convert the QUBO model to a Binary Quadratic Model (BQM)
    bqm = model.to_bqm()

    # Select the sampler: Use quantum annealing if `quantum=True`, otherwise use simulated annealing
    if quantum:
        # Quantum annealing using D-Wave's quantum sampler with embedding composite
        sampler = EmbeddingComposite(DWaveSampler())
    else:
        # Simulated annealing using a classical solver from the 'neal' library
        sampler = neal.SimulatedAnnealingSampler()

    # Sample from the BQM with 1000 reads (samples) to find the best solution
    sampleset = sampler.sample(bqm, num_reads=1000)

    # Extract the first (best) solution from the sampleset
    sample = sampleset.first

    return sample.sample, sample.energy

In [49]:
simulated_annealing_sample, simulated_annealing_energy = solve_qubo_annealing(bpp_qubo_model)
quantum_annealing_sample, quantum_annealing_energy = solve_qubo_annealing(bpp_qubo_model, 1)

In [50]:
print(parse_qubo_solution(simulated_annealing_sample, weights_small))
print(f"Minimum energy of simulated annealing: {simulated_annealing_energy}")
print(parse_qubo_solution(quantum_annealing_sample, weights_small))
print(f"Minimum energy of quantum annealing: {quantum_annealing_energy}")

Minimum number of bins used: 2
Bin 1 contains items: item 1 (weight 8) item 2 (weight 2) 
Bin 2 contains items: item 3 (weight 3) item 4 (weight 7) 
None
Minimum energy of simulated annealing: 2.0
Minimum number of bins used: 2
Bin 2 contains items: item 1 (weight 8) item 2 (weight 2) 
Bin 4 contains items: item 3 (weight 3) item 4 (weight 7) 
None
Minimum energy of quantum annealing: 2.0


# Section 3: Solving Ising Models with Variational Techniques

In this section, we attempt to solve the Bin Packing Problem (BPP) using a variational technique. Specifically, we use the Variational Quantum Eigensolver (VQE) to address the problem.

Since VQE computes the smallest eigenvalue of an Ising Hamiltonian, the first step is to translate the BPP into an Ising model. Fortunately, PyQUBO provides a built-in function for this translation. However, the Ising model produced by PyQUBO is not directly compatible with Qiskit. To overcome this, we define a helper function that converts the PyQUBO Ising model into a format that is compatible with Qiskit.

In [51]:
bpp_qubo = get_qubo(weights_small, bin_capacity_small)

In [52]:
from qiskit.quantum_info import SparsePauliOp, Pauli

In [53]:
def pyqubo_to_qiskit_ising(qubo):
    """
    Converts a PyQUBO QUBO model into a Qiskit-compatible Ising Hamiltonian.

    Args:
        qubo (pyqubo.PyQUBO): The QUBO model created using PyQUBO.

    Returns:
        tuple: A tuple containing:
            - A Qiskit SparsePauliOp representing the Ising Hamiltonian.
            - The offset term from the QUBO.

    This function performs the following steps:
    1. Translates the QUBO variables into qubit indices.
    2. Extracts the linear and quadratic terms from the QUBO model.
    3. Converts the linear terms into Pauli-Z operators for each qubit.
    4. Converts the quadratic terms into Pauli-Z operators acting on pairs of qubits.
    5. Returns the Ising Hamiltonian as a Qiskit SparsePauliOp and the offset.
    """

    # Number of qubits corresponds to the number of variables in the QUBO model
    num_qubits = len(qubo.variables)

    # Map the QUBO variable names to qubit indices
    term_to_qubit = dict(zip(qubo.variables, [i for i in range(num_qubits)]))

    # Extract the linear terms, quadratic terms, and offset from the QUBO
    linears = qubo.to_ising()[0]  # Linear terms
    quadratics = qubo.to_ising()[1]  # Quadratic terms
    offset = qubo.to_ising()[2]  # Offset term

    # Initialize a list to hold Pauli terms (Pauli strings) for the Hamiltonian
    pauli_list = []

    # Convert linear terms to Pauli-Z operators (acting on a single qubit)
    for term in linears.keys():
        weight = linears[term]
        z_string = ['I'] * num_qubits  # Start with identity ('I') for all qubits
        i = term_to_qubit[term]  # Get the qubit index for this term
        z_string[i] = 'Z'  # Replace identity with Pauli-Z operator for this qubit
        pauli_list.append((weight, ''.join(z_string)))  # Add the Pauli-Z term with its weight

    # Convert quadratic terms to Pauli-Z operators (acting on pairs of qubits)
    for term in quadratics.keys():
        weight = quadratics[term]
        z_string = ['I'] * num_qubits  # Start with identity for all qubits
        i = term_to_qubit[term[0]]  # Get the qubit index for the first variable in the pair
        j = term_to_qubit[term[1]]  # Get the qubit index for the second variable in the pair
        z_string[i] = 'Z'  # Apply Pauli-Z to the first qubit
        z_string[j] = 'Z'  # Apply Pauli-Z to the second qubit
        pauli_list.append((weight, ''.join(z_string)))  # Add the Pauli-Z pair term with its weight

    # Extract the coefficients and Pauli strings for the Hamiltonian
    coeffs = [term[0] for term in pauli_list]
    paulis = [term[1] for term in pauli_list]

    # Construct the Qiskit Ising Hamiltonian as a SparsePauliOp
    hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

    # Return the Hamiltonian and the offset term
    return hamiltonian, offset

Once the Ising model is made, we then define a base quantum ansatz. This ansatz will be optimized for low eigenvalue (best bin value solution) and will tune the circuit parameters towards the best bin packing assignment.

Using Qiskit circuits, we create functions to both prepare an ansatz and use VQE to solve for the optimal bin assigment.

Our chosen ansatz is the Hardware-Efficient Ansatz (HEA) which is frequently used for VQE. We also provide a layered ansatz and ucc ansatz which are also frequently used although in different contexts. We do our testing with HEA.

Our VQE implementation uses the classical COBYLA optimizer which is popular due to its fast convergence, low memory, and flexibility on high dimensional spaces.

In [54]:
from qiskit.circuit import QuantumCircuit, Parameter

In [55]:
def create_hea_ansatz(num_qubits, depth=1):
    """
    Creates a parameterized quantum circuit (ansatz) consisting of single-qubit
    rotations and entangling gates (CNOTs). The circuit is designed for use in
    variational quantum algorithms like the Variational Quantum Eigensolver (VQE).

    The ansatz consists of a series of parameterized single-qubit rotations
    (RX gates) followed by a layer of CNOT gates to introduce entanglement.
    The depth parameter controls how many times this structure is repeated.

    Args:
        num_qubits (int): The number of qubits in the quantum circuit.
        depth (int, optional): The depth of the circuit. A depth of 1 means
                               only one layer of single-qubit rotations and
                               CNOTs. Default is 1.

    Returns:
        QuantumCircuit: A quantum circuit with the specified number of qubits
                        and parameterized gates.
        list: A list of Parameters (θ_i) representing the variational parameters.
    """
    # Create an empty quantum circuit with the specified number of qubits
    qc = QuantumCircuit(num_qubits)

    # Initialize a list of parameters for each qubit at each depth level
    # The number of parameters will be num_qubits * depth
    params = [Parameter(f"θ_{i}") for i in range(num_qubits * depth)]

    count = 0  # Counter to track the current parameter

    # Loop over the depth of the ansatz
    for _ in range(depth):

        # Apply RX rotations (parameterized single-qubit rotations) to each qubit
        for qubit in range(num_qubits):
            qc.rx(params[count], qubit)  # Apply RX gate with the current parameter
            count += 1  # Move to the next parameter

        # Apply CNOT gates (entangling gates) between consecutive qubits
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)  # Apply CNOT between qubit[i] and qubit[i+1]

    # Return the quantum circuit and the list of parameters
    return qc, params

In [ ]:
def create_layered_ansatz(n_qubits, depth=1):
    """
    Creates a layered ansatz circuit for quantum optimization problems.

    This function constructs a parameterized quantum circuit with alternating
    RX and RY rotations applied to each qubit, followed by CNOT gates to entangle
    the qubits. The ansatz is repeated for the specified number of layers (depth).

    Args:
        n_qubits (int): The number of qubits in the quantum circuit.
        depth (int): The number of layers of parameterized gates to apply.

    Returns:
        tuple: A tuple containing:
            - `qc` (QuantumCircuit): The parameterized quantum circuit.
            - `params` (list): A list of parameters associated with the gates.
    """
    # Initialize the quantum circuit with the given number of qubits
    qc = QuantumCircuit(n_qubits)

    # Create a list of parameters for RX and RY gates, one for each qubit in each layer
    params = [Parameter(f"theta_{i}") for i in range(n_qubits * depth * 2)]

    # Initialize the parameter index to keep track of the next parameter to use
    param_index = 0

    # Apply the ansatz layers
    for _ in range(depth):
        # Apply RX and RY rotations to each qubit in the current layer
        for i in range(n_qubits):
            qc.rx(params[param_index], i)  # RX rotation on qubit i
            qc.ry(params[param_index + 1], i)  # RY rotation on qubit i
            param_index += 2  # Move to the next pair of parameters

        # Apply CNOT gates between consecutive qubits to create entanglement
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)

    # Return the quantum circuit and the list of parameters
    return qc, params

In [ ]:
def creat_ucc_ansatz(n_qubits, depth):
    """
    Creates a Unitery Coupled-Cluster (UCC) ansatz for quantum chemistry simulations.

    This ansatz is commonly used in the Variational Quantum Eigensolver (VQE) algorithm
    for quantum chemistry. It consists of applying parameterized RX rotations to each qubit,
    followed by entangling CNOT gates between consecutive qubits. The number of layers (depth)
    determines the expressiveness of the ansatz.

    Args:
        n_qubits (int): The number of qubits in the quantum circuit.
        depth (int): The number of layers (depth) for the ansatz.

    Returns:
        tuple: A tuple containing:
            - `qc` (QuantumCircuit): The parameterized quantum circuit.
            - `params` (list): A list of parameters associated with the RX gates.
    """

    # Initialize a quantum circuit with the specified number of qubits
    qc = QuantumCircuit(n_qubits)

    # Create a list of parameters for RX gates, one for each qubit in each layer
    params = [Parameter(f"theta_{i}") for i in range(n_qubits * depth)]

    # Apply the ansatz for the specified number of layers (depth)
    for d in range(depth):
        # Apply an RX rotation to each qubit in the current layer
        for i in range(n_qubits):
            qc.rx(params[d * n_qubits + i], i)  # Parameterized RX rotation on qubit i

        # Apply CNOT gates between consecutive qubits to entangle them
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)  # CNOT gate between qubit i and qubit i+1

    # Return the quantum circuit and the list of parameters
    return qc, params

In [56]:
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator

In [57]:
def solve_with_vqe(qubo, ansatz_depth=1, ansatz_choice=0):
    """
    Solves a Quadratic Unconstrained Binary Optimization (QUBO) problem using the
    Variational Quantum Eigensolver (VQE) algorithm. The QUBO is first converted to
    an Ising Hamiltonian, and then the VQE algorithm is used to find the optimal
    solution. The solution is sampled using a quantum simulator.

    Args:
        qubo (pyqubo.QUBO): The QUBO problem to solve, represented using the pyqubo
                             package.
        ansatz_depth (int, optional): The depth of the variational quantum ansatz.
                                       A higher depth means more layers of gates in
                                       the quantum circuit. Default is 1.

    Returns:
        tuple: A tuple containing:
            - best_solution_vector (np.ndarray): The best binary solution found by
                                                  the VQE algorithm, represented as
                                                  a NumPy array.
            - energy (float): The minimum energy corresponding to the best solution,
                              including any offset from the QUBO to Ising conversion.
    """

    # Step 1: Convert the QUBO problem into an Ising Hamiltonian and get the offset
    hamiltonian, offset = pyqubo_to_qiskit_ising(qubo)

    # Step 2: Define the ansatz (variational quantum circuit)
    num_qubits = len(qubo.variables)  # The number of qubits is the number of QUBO variables

    ansatz = None
    params = None

    if ansatz_choice == 0:
        ansatz, params = create_hea_ansatz(num_qubits, ansatz_depth)  # Create the quantum circuit ansatz
    elif ansatz_choice == 1:


    # Step 3: Set up the Estimator and VQE solver
    backend = AerSimulator()  # Use a quantum simulator as the backend for running the circuit
    estimator = Estimator()  # Create an Estimator to compute the expectation values
    vqe_solver = VQE(estimator, ansatz, optimizer=COBYLA())  # Set up the VQE solver with COBYLA optimizer

    # Step 4: Run VQE to compute the minimum eigenvalue of the Ising Hamiltonian
    result = vqe_solver.compute_minimum_eigenvalue(operator=hamiltonian)  # Compute the energy

    # Step 5: Construct the quantum circuit with the optimal parameters found by VQE
    sampled_circuit = ansatz.assign_parameters({param: value for param, value in zip(params, result.optimal_point)})
    sampled_circuit.measure_all()  # Add measurement to all qubits

    # Step 6: Run the circuit on a quantum simulator
    job = backend.run(sampled_circuit, shots=1024)  # Run the circuit with 1024 shots (measurements)
    counts = job.result().get_counts()  # Get the measurement results

    # Step 7: Extract the best solution from the measurement results
    best_solution_str = max(counts, key=counts.get)  # Find the most frequent outcome (best solution)
    best_solution_vector = np.array([int(bit) for bit in best_solution_str[::-1]])  # Convert to binary vector

    # Step 8: Return the best solution and the corresponding energy (including the offset)
    return best_solution_vector, result.eigenvalue.real + offset

In [58]:
bin_assignment, val = solve_with_vqe(bpp_qubo)

<ipython-input-57-bf97fc367962>:33: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()  # Create an Estimator to compute the expectation values


In [59]:
print("VQE solution to BPP: ", bin_assignment)

VQE solution to BPP:  [1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1]


# Section 4: Solving QUBOs with Quantum Approximate Optimization Algorithm

To solve a QUBO problem using the Quantum Approximate Optimization Algorithm (QAOA), we follow a structured approach inspired by prior work with QAOA. Based on my experience, the most efficient strategy involves converting the PyQUBO model into a Qiskit QuadraticProgram format. This conversion is essential because it allows us to leverage Qiskit's powerful tools for optimization and quantum circuit construction.

Once the QuadraticProgram is obtained, we proceed to build the QAOA circuit by defining several key components, including:

*   Custom Cost Hamiltonian: Represents the problem's objective function.
*   Mixer Hamiltonian: Facilitates exploration of the solution space.
*   Expectation function: Measures the expected outcome of the quantum process.

In the process of building the QAOA circuit, I also define two key helper functions: get_qp and get_qubo_matrix. While these functions share some overlap with previous QUBO models and with each other, they are essential for ensuring that the QUBO is represented in the appropriate form for QAOA. These dual representations are important because QAOA requires multiple views of the same QUBO model to interact effectively with different parts of the quantum optimization process.

In [60]:
from qiskit_optimization.translators import from_docplex_mp

In [61]:
def get_qp(weights, bin_capacity, single_assignment_penalty=10, bin_capacity_penalty=10):
    """
    Converts the given Bin Packing Problem (BPP) model to a Qiskit Quadratic Program (QP) format.

    Args:
        model (pyomo.Model): The original BPP model.
        weights (list): A list of item weights.
        bin_capacity (int): The maximum capacity of each bin.
        single_assignment_penalty (int, optional): The penalty applied for violating the constraint that each item
            must be assigned to exactly one bin. Default is 10.
        bin_capacity_penalty (int, optional): The penalty applied for violating the bin capacity constraint.
            Default is 10.

    Returns:
        QuadraticProgram: A Qiskit Quadratic Program (QP) representing the BPP.

    The function constructs a QP by:
    1. Defining binary decision variables for assigning items to bins.
    2. Defining binary variables to represent whether a bin is used.
    3. Setting an objective to minimize the number of bins used.
    4. Adding constraints for each item to be assigned to exactly one bin.
    5. Adding constraints to ensure that bin capacity is not exceeded.
    """

    # Number of items and bins (in the worst case, one bin per item)
    n_items = len(weights)
    n_bins = n_items

    # Create a new Pyomo model for this QP conversion
    model = Model()

    # Define binary variables x[i][j] where x[i][j] is 1 if item i is placed in bin j, 0 otherwise
    x = {(i, j): model.binary_var(name=f"x_{i}_{j}") for i in range(n_items) for j in range(n_bins)}

    # Define binary variables y[j] where y[j] is 1 if bin j is used, 0 otherwise
    y = {j: model.binary_var(name=f"y_{j}") for j in range(n_bins)}

    # Objective function: Minimize the number of bins used (sum of y[j] over all bins)
    H_obj = sum(y[j] for j in range(n_bins))

    # Constraint 1: Each item must be assigned to exactly one bin
    H_single_assignment = sum((sum(x[(i, j)] for j in range(n_bins)) - 1)**2 for i in range(n_items))

    # Constraint 2: Bin capacity constraint
    H_capacity = sum((sum(weights[i] * x[(i, j)] for i in range(n_items)) - bin_capacity * y[j])**2 for j in range(n_bins))

    # Combine the objective and constraints into one Hamiltonian
    # The penalties are applied to ensure that violations of constraints are heavily penalized
    H = H_obj + single_assignment_penalty * H_single_assignment + bin_capacity_penalty * H_capacity

    # Minimize the Hamiltonian to form the QP
    model.minimize(H)

    # Convert the Pyomo model to a Qiskit Quadratic Program (QP) format
    qp = from_docplex_mp(model)

    return qp

In [80]:
import numpy as np

def get_qubo_matrix(weights, bin_capacity, penalty_weight=10):
    """
    Generates the QUBO (Quadratic Unconstrained Binary Optimization) matrix for the bin packing problem.

    This matrix is used to represent the bin packing problem as a quadratic optimization problem,
    where the objective is to minimize the number of bins used while respecting bin capacity constraints
    and ensuring that each item is assigned to exactly one bin.

    Args:
        weights (list): A list of item weights to be packed into bins.
        bin_capacity (float): The maximum capacity of a single bin.
        penalty_weight (float, optional): A weight used to penalize constraints that are violated (default is 10).

    Returns:
        np.ndarray: A square matrix representing the QUBO formulation of the bin packing problem.
    """

    num_items = len(weights)  # Number of items to pack
    num_bins = num_items  # Worst case: one bin per item (can be relaxed later)

    total_vars = num_bins * num_items + num_bins  # Total number of binary variables (x_ij and y_j)
    Q = np.zeros((total_vars, total_vars))  # Initialize the QUBO matrix as a zero matrix

    # Helper function to compute the index of x_ij (item i assigned to bin j)
    def x_index(i, j):
        return i * num_bins + j

    # Helper function to compute the index of y_j (bin j is used)
    def y_index(j):
        return num_items * num_bins + j

    # Objective: Minimize the number of bins used (minimize y_j)
    for j in range(num_bins):
        # Add penalty for using bin j (we want to minimize the number of bins used)
        Q[y_index(j), y_index(j)] += penalty_weight

    # Single-bin assignment constraint: Ensure each item is assigned to exactly one bin
    for i in range(num_items):
        for j in range(num_bins):
            # Subtract penalty for assigning item i to bin j (since this is part of the objective)
            Q[x_index(i, j), x_index(i, j)] -= penalty_weight

        # Add penalties to enforce exactly one bin per item (no double assignments)
        for j in range(num_bins):
            for k in range(j + 1, num_bins):
                # If item i is assigned to both bin j and bin k, this is a violation
                Q[x_index(i, j), x_index(i, k)] += 2 * penalty_weight

    # Link x_ij and y_j: If item i is assigned to bin j, then bin j must be used (y_j = 1)
    for i in range(num_items):
        for j in range(num_bins):
            # If x_ij is 1, then y_j must also be 1. Thus, we enforce this link.
            Q[x_index(i, j), y_index(j)] -= penalty_weight
            Q[y_index(j), y_index(j)] += penalty_weight

    # Capacity constraint: Penalize if the combined weight of items in a bin exceeds the bin's capacity
    for j in range(num_bins):
        for i in range(num_items):
            for k in range(i + 1, num_items):
                # If the combined weight of items i and k exceeds the bin capacity, we add a penalty
                if weights[i] + weights[k] > bin_capacity:
                    Q[x_index(i, j), x_index(k, j)] += penalty_weight * (weights[i] + weights[k] - bin_capacity)

    return Q

I finally define the BPP_QAOA class. This is a custom designed QAOA procedure for the bin packing problem. The architecture is the same proposed by (Farhi et al. 2014) and is nicely described in this video:

https://www.youtube.com/watch?v=YpLzSQPrgSc&t=1129s

Similar to VQE, we chose to use the Cobyla optimizer due to its fast convergence speed, flexibilty over high dimensional spaces, and low memory usage. Further details for the QAOA implementation and details are discussed in the docstrings and comments.

In [81]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.visualization import plot_histogram, circuit_drawer
from qiskit import transpile, assemble
from scipy.optimize import minimize
from qiskit_aer import Aer

from collections import defaultdict, Counter

import math

In [100]:
class BPP_QAOA:
    """
    A class that implements the QAOA (Quantum Approximate Optimization Algorithm) for solving the Bin Packing Problem (BPP).

    This class is designed to formulate and solve the bin packing problem using a quantum computer.
    It transforms the problem into a QUBO (Quadratic Unconstrained Binary Optimization) form and uses
    QAOA to approximate the optimal solution.

    Attributes:
        weights (list): A list of item weights to be packed into bins.
        bin_capacity (float): The maximum capacity of a single bin.
        converter (QuadraticProgramToQubo): A converter that transforms a QuadraticProgram into a QUBO problem.
        qp_qubo (QuadraticProgram): The QUBO problem corresponding to the bin packing problem.
        qubo (np.ndarray): The QUBO matrix representing the problem.
        backend (Backend): The quantum backend used for execution.
        circuit (QuantumCircuit): The quantum circuit representing the QAOA ansatz.
        depth (int): The number of layers (depth) for the QAOA circuit.
        num_qubits (int): The number of qubits required to represent the QUBO problem.
        quadratics (dict): The quadratic terms of the QUBO problem.
        linears (dict): The linear terms of the QUBO problem.
        constant (float): The constant term of the QUBO problem.
    """

    def __init__(self, qp, weights, bin_capacity, depth=1, backend=Aer.get_backend('qasm_simulator')):
        """
        Initializes the BPP_QAOA class.

        Args:
            qp (QuadraticProgram): The quadratic program formulation of the bin packing problem.
            weights (list): A list of item weights to be packed into bins.
            bin_capacity (float): The capacity of a single bin.
            depth (int, optional): The depth of the QAOA circuit. Default is 1.
            backend (Backend, optional): The quantum backend for the execution. Default is the QASM simulator.
        """
        # Store the problem data
        self.weights = weights
        self.bin_capacity = bin_capacity

        # Convert the quadratic program (QP) to QUBO form
        self.converter = QuadraticProgramToQubo()
        self.qp_qubo = self.converter.convert(qp)

        # Generate the QUBO matrix based on weights and bin capacity
        self.qubo = get_qubo_matrix(self.weights, self.bin_capacity)

        # Store the backend and other variables
        self.backend = backend
        self.circuit = None
        self.depth = depth
        self.num_qubits = self.qp_qubo.get_num_vars()

        # Convert the quadratic and linear terms of the QUBO to dictionaries
        self.quadratics = dict(self.qp_qubo.objective.quadratic.to_dict())
        self.linears = dict(self.qp_qubo.objective.linear.to_dict())
        self.constant = self.qp_qubo.objective.constant

    def cost_H(self, gamma, quadratics, linears):
        """
        Constructs the cost Hamiltonian (the operator that encodes the objective function of the problem).

        This Hamiltonian is responsible for encoding the QUBO quadratic and linear terms
        into quantum operations. It uses `RZ` gates for linear terms and `RZZ` gates for quadratic terms.

        Args:
            gamma (float): The QAOA parameter for the cost Hamiltonian.
            quadratics (dict): The quadratic terms of the QUBO problem.
            linears (dict): The linear terms of the QUBO problem.

        Returns:
            QuantumCircuit: The quantum circuit representing the cost Hamiltonian.
        """
        qc = QuantumCircuit(self.num_qubits)

        # Apply linear terms as RZ rotations
        for i in range(self.num_qubits):
            linear_term = linears.get(i, 0)
            quadratic_term = sum(quadratics.get((i, j), 0) for j in range(self.num_qubits))
            qc.rz(0.5 * (linear_term + quadratic_term) * gamma, i)

        # Apply quadratic terms as RZZ rotations for each pair of qubits
        for (i, j), coeff in quadratics.items():
            if i != j:
                qc.rzz(0.25 * coeff * gamma, i, j)

        qc.barrier()  # Add a barrier to separate sections of the circuit
        return qc

    def mixer_H(self, beta):
        """
        Constructs the mixer Hamiltonian (the operator that encodes the mixing operations between states).

        This Hamiltonian is used to mix the quantum states and explore different configurations. It
        uses `RX` gates for the mixer.

        Args:
            beta (float): The QAOA parameter for the mixer Hamiltonian.

        Returns:
            QuantumCircuit: The quantum circuit representing the mixer Hamiltonian.
        """
        qc = QuantumCircuit(self.num_qubits)
        for i in range(self.num_qubits):
            qc.rx(2 * beta, i)  # Apply RX rotations to each qubit
        qc.barrier()  # Add a barrier to separate sections of the circuit
        return qc

    def qaoa_ansatz(self, gammas, betas, quadratics, linears):
        """
        Constructs the QAOA ansatz circuit (the quantum circuit representing the alternating sequence
        of cost and mixer Hamiltonians).

        The ansatz applies the cost Hamiltonian and the mixer Hamiltonian alternately
        for the specified number of layers (depth).

        Args:
            gammas (list): The list of gamma parameters for the cost Hamiltonian.
            betas (list): The list of beta parameters for the mixer Hamiltonian.
            quadratics (dict): The quadratic terms of the QUBO problem.
            linears (dict): The linear terms of the QUBO problem.

        Returns:
            QuantumCircuit: The complete QAOA circuit.
        """
        circuit = QuantumCircuit(self.num_qubits, self.num_qubits)
        assert len(betas) == len(gammas)  # Ensure matching lengths for gammas and betas
        p = len(betas)  # Number of layers (depth)

        circuit.h(range(self.num_qubits))  # Apply Hadamard gate to all qubits to create superposition
        circuit.barrier()  # Add a barrier to separate sections of the circuit

        # Apply alternating layers of cost and mixer Hamiltonians
        for i in range(p):
            circuit.compose(self.cost_H(gammas[i], quadratics, linears), inplace=True)
            circuit.compose(self.mixer_H(betas[i]), inplace=True)

        circuit.measure(range(self.num_qubits), range(self.num_qubits))  # Measure the qubits
        return circuit

    def optimize_qaoa(self, initial_params=None):
        """
        Optimizes the QAOA parameters using classical optimization.

        The method minimizes the expectation value of the QUBO Hamiltonian by adjusting the
        QAOA parameters (gammas and betas) using the COBYLA optimization algorithm.

        Args:
            initial_params (list, optional): The initial parameters for optimization. If None, defaults to [0.5] * (2 * depth).

        Returns:
            None: The optimized parameters are used to generate the final QAOA circuit.
        """
        if initial_params is None:
            initial_params = [0.5] * (2 * self.depth)  # Default to 0.5 for all parameters

        # Define the expectation value function for QAOA
        def bpp_expectation(theta):
            middle = len(theta) // 2
            gammas = theta[:middle]
            betas = theta[middle:]
            qc = self.qaoa_ansatz(gammas, betas, self.quadratics, self.linears)

            # Run the quantum circuit on the backend and get the results
            job = self.backend.run(transpile(qc, self.backend), shots=1024)
            counts = job.result().get_counts()

            # Calculate the energy of the system based on the QUBO matrix
            energy = 0
            for bitstring, count in counts.items():
                z = np.array([1 if bit == '1' else -1 for bit in bitstring[::-1]])
                energy += count * z.T @ self.qubo @ z

            return energy / 1024  # Normalize the energy by the number of shots

        # Minimize the expectation function using COBYLA optimizer
        res = minimize(bpp_expectation, initial_params, method='COBYLA')

        # Split the results into gammas and betas
        middle = len(res.x) // 2
        gammas = res.x[:middle]
        betas = res.x[middle:]

        # Generate the final QAOA circuit using the optimized parameters
        self.circuit = self.qaoa_ansatz(gammas, betas, self.quadratics, self.linears)

    def run_circuit(self):
        """
        Executes the optimized QAOA circuit on the quantum backend and returns the results.

        The circuit is transpiled for the selected backend, executed with a shot count of 1024,
        and the 40 most common results are returned.

        Returns:
            dict: A dictionary containing the 40 most common measurement results.
        """
        transpiled_circuit = transpile(self.circuit, self.backend)
        job = self.backend.run(transpiled_circuit, shots=1024)
        counts = job.result().get_counts()
        most_common_results = dict(Counter(counts).most_common(40))  # Get the 40 most common results

        def find_most_common_solutions(input_dict, n):
            sorted_keys = sorted(input_dict, key=input_dict.get, reverse=True)
            return sorted_keys[:n]

        most_common_bitstring = find_most_common_solutions(counts, 1)

        return most_common_results, most_common_bitstring

In [101]:
bpp_qp = get_qp(weights_small, bin_capacity_small)
bpp_qaoa = BPP_QAOA(bpp_qp, weights_small, bin_capacity_small)

In [112]:
bpp_qaoa.optimize_qaoa()
qaoa_histogram_results, qaoa_bpp_solution = bpp_qaoa.run_circuit()

In [113]:
print("QAOA solution to BPP: " + str(qaoa_bpp_solution))

QAOA solution to BPP: ['01100111110111001000']


# Section 5: Comparisons and Conclusions

## Analysis of Bin Packing Solutions for Small, Medium, and Large Instances
We evaluated the performance of various algorithms for solving bin packing problems of different sizes—small, medium, and large. These tests were done seperately to avoid notebook clutter. The results highlight how these methods scale as the problem size increases and provide insights into their strengths and weaknesses.

### Small Instance Results
**Integer Linear Programming (ILP)**: ILP found the optimal solution. This configuration was optimal and met all capacity constraints. The solution was computed quickly and accurately.

**Brute Force**: Brute force also identified the optimal solution, confirming the correctness of the ILP result. Since the problem was small, brute force performed well, but it’s worth noting that its time complexity grows exponentially, making it impractical for larger instances.

**D-Wave Quantum Annealing**: D-Wave produced the same optimal solution as ILP and brute force, demonstrating that the quantum annealer could effectively converge to the best solution for this small problem instance. The quantum annealer was able to handle the problem efficiently and quickly, matching the classical methods.

**VQE (Variational Quantum Eigensolver)**: VQE provided a suboptimal solution with a higher value (less negative) than the optimal solution found by ILP, brute force, and D-Wave. This suggests that VQE had difficulty converging to the best configuration for this instance, potentially due to limitations in its parameter optimization or circuit depth.

**QAOA (Quantum Approximate Optimization Algorithm)**: QAOA performed better than VQE but did not achieve the optimal solution found by ILP or D-Wave. The solution was close to optimal but still slightly off, possibly due to the depth of the quantum circuit or suboptimal parameter choices.

### Key Takeaways for Small Instances:

ILP and brute force provide reliable, optimal solutions.
D-Wave demonstrated the potential of quantum annealing to solve small combinatorial problems optimally and efficiently.
VQE and QAOA performed less optimally, potentially due to issues with parameter optimization, circuit depth, or noise.

### Medium Instance Results
**Integer Linear Programming (ILP)**: ILP was able to find the optimal solution, though it took longer than for the small instance. As the number of items increased, ILP required more processing time, but it still produced an optimal configuration and met all capacity constraints in a reasonable time frame.

**Brute Force**: Brute force was still able to find the optimal solution, but it became noticeably slower as the problem size grew. The exponential time complexity of brute force made it less practical for medium-sized problems, although it was still usable for this instance. The increased computation time highlighted the scalability issues of brute force.

**D-Wave Quantum Annealing**: D-Wave continued to perform well for the medium-sized instance, providing an optimal solution in a faster time than brute force and ILP. The quantum annealer demonstrated its efficiency at this scale, handling the increased complexity without running into memory issues or excessive computation times.

**VQE (Variational Quantum Eigensolver)**: VQE showed some improvement over the small instance, but it still struggled to reach the optimal solution. The solution was close but had a higher value than the ILP result, indicating that VQE was unable to find the best configuration. The faster convergence compared to the small instance was a sign of progress, but more refinement in the algorithm was needed for reliable optimal results at this scale.

**QAOA (Quantum Approximate Optimization Algorithm)**: QAOA performed better than VQE in this case, providing a solution that was closer to optimal. It outperformed VQE in terms of solution quality and speed, although it still did not achieve the optimal solution. QAOA benefited from a better circuit depth or parameter setup compared to the small case but was still an approximate method and could not guarantee optimal results.

### Key Takeaways for Medium Instances:

ILP and brute force still provide optimal solutions, though brute force is less practical due to increased computation time.
D-Wave proved to be an efficient method, solving the problem quickly while maintaining accuracy.
VQE and QAOA performed better than in the small instance, with QAOA performing closer to optimal. However, both still struggled to guarantee optimal solutions, indicating that further optimization and deeper circuits may be needed for larger medium-sized problems.

### Large Instance Results
**Integer Linear Programming (ILP)**: ILP was able to find the optimal solution, but the time required to process the larger instance grew significantly. While ILP can guarantee an optimal solution, the computational resources and time required to solve large instances become a limitation. The solver still produced correct solutions, but as the problem size grows, the time required for solution increases dramatically due to the exponential complexity of solving linear programming problems with many variables and constraints.

**Brute Force**: Brute force, while guaranteed to find the optimal solution, took an excessive amount of time due to the exponential growth in the number of combinations that needed to be evaluated. The time required to complete the solution made brute force impractical for large instances, highlighting the scalability challenges of this method.

**VQE (Variational Quantum Eigensolver)**: VQE encountered a memory error during execution, likely due to the complexity of the problem. The memory required to store the quantum state space and process the Hamiltonian grew exponentially with the number of qubits, causing the algorithm to fail for larger instances. This points to the need for further development in quantum algorithms to handle larger problem spaces efficiently.

**QAOA (Quantum Approximate Optimization Algorithm)**: QAOA also encountered a memory error, similar to VQE. The increasing complexity of the quantum circuit required too many resources, and the backend could not handle the computation. This suggests that the circuit complexity was too high for the available resources, limiting the performance of QAOA on large instances.

**D-Wave Quantum Annealing**: D-Wave was the only method that successfully found an optimal solution. It did so efficiently, without encountering memory issues or excessive computation times. The quantum annealer was able to handle the larger problem instance, showcasing its scalability and resource efficiency in solving combinatorial optimization problems at large scales.

## Conclusion
In summary, D-Wave quantum annealing emerged as the most robust and scalable solution, effectively handling both small and large bin packing problems, while classical methods like ILP and Brute Force were reliable for smaller instances but faced scalability issues. Quantum methods like VQE and QAOA demonstrated potential but are not yet capable of consistently solving large-scale combinatorial optimization problems without further refinement.